In [1]:
import requests
import pandas as pd

from datetime import date
from io import BytesIO

from datetime import datetime
from dateutil.parser import parse
from tqdm import tqdm

import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("KMTC")

In [4]:
bl_list 

['KMTCJKT4653697',
 'KMTCJKT4654708',
 'KMTCJKT4659713',
 'KMTCJKT4677895',
 'KMTCJKT4670989',
 'KMTCJKT4681560',
 'KMTCJKT4699130',
 'KMTCJKT4680636']

In [5]:
# array for failed tracking
gagal=[]

In [6]:
hasil_akhir = []
for i, bls in enumerate(tqdm(bl_list)):
    try:
        # hitting the API
        url_ctr = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}?dtKnd=BL&blNo={}".format(
            bls[4:], bls[4:])
        response_ctr = requests.request("GET", url_ctr)

        # getting container data. booking id needed for API URL
        ctr_data = response_ctr.json()
        list_of_container_number = []
        list_of_bookingid = []

        for j, ctr_info in enumerate(ctr_data['cntrList']):
            list_of_container_number.append(ctr_info["cntrNo"])
            list_of_bookingid.append(ctr_info["bkgNo"])

        for c, ctr in enumerate(list_of_container_number):
            url_milestone = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}/detail?bkgNo={}&cntrNo={}&dtKnd=BL&strBkgNo={}".format(
                bls[4:], list_of_bookingid[c], ctr, list_of_bookingid[c])
            
            # hitting the API again for every container for container milestones
            response_milestone = requests.request("GET", url_milestone)

            # aqcuiring milestone datas for every container
            milstone_data = response_milestone.json()
            current_dict = {}
            dict_milestone = {}
            milestones = []
            key_mapping = {
                'cntrNo': 'Container Number',
                'etd': 'ETD',
                'eta': 'ETA',
                # 'GTOOB':'Gate Out Origin',
                # 'GTIOB':'Gate In Origin',
                'LDGOB': 'ATD',
                'DISIB': 'ATA',
                'GTOIB': 'Container Release',
                'GTIIB': 'Container Return'
            }

            current_dict.update({"Liners": "KMTC"})
            current_dict.update({"BL Number": bls})
            current_dict.update(
                {"From": milstone_data['trackingList'][-1]['plcNm'][:milstone_data['trackingList'][-1]['plcNm'].index(",")]})
            current_dict.update(
                {"To": milstone_data['trackingList'][-1]['podPortNm']})

            for key, label in key_mapping.items():
                if key in ctr_data['cntrList'][c]:
                    if key == "etd" or key == "eta":
                        current_dict[label] = datetime.strptime(
                            ctr_data['cntrList'][c][key][:8], "%Y%m%d").strftime("%Y-%m-%d")
                    else:
                        current_dict[label] = ctr_data['cntrList'][c][key]

            for b, milestone in enumerate(milstone_data['trackingList']):
                milestones.append(
                    milestone["cntrStsCd"] + milestone["cntrMvntCd"])
                milestones.append(milestone["mvntDt"])

            for m, milestone in enumerate(milestones):
                if is_date(milestone):
                    case_milestone = {
                        milestones[m-1]: datetime.strptime(milestone, "%Y%m%d").strftime("%Y-%m-%d")}
                    dict_milestone.update(case_milestone)

            for key, label in key_mapping.items():
                if key in dict_milestone:
                    current_dict[label] = dict_milestone[key]

            print(current_dict)
            hasil_akhir.append(current_dict)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)


  0%|          | 0/8 [00:00<?, ?it/s]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4653697', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'BMOU6920374', 'ETD': '2023-11-22', 'ETA': '2023-11-25', 'ATD': '2023-11-22', 'ATA': '2023-12-09', 'Container Release': '2023-12-19', 'Container Return': '2023-12-22'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4653697', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'KMTU9354174', 'ETD': '2023-11-22', 'ETA': '2023-11-25', 'ATD': '2023-11-22', 'ATA': '2023-12-09', 'Container Release': '2023-12-19', 'Container Return': '2023-12-22'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4653697', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'SEGU4808737', 'ETD': '2023-11-22', 'ETA': '2023-11-25', 'ATD': '2023-11-22', 'ATA': '2023-12-09', 'Container Release': '2023-12-19', 'Container Return': '2023-12-22'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4653697', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'SEGU6460595', 'ETD': '2023-11-22', 'ETA': '2023-11-

 12%|█▎        | 1/8 [00:02<00:16,  2.37s/it]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4653697', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'TEMU8988124', 'ETD': '2023-11-22', 'ETA': '2023-11-25', 'ATD': '2023-11-22', 'ATA': '2023-12-09', 'Container Release': '2023-12-19', 'Container Return': '2023-12-22'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4654708', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'FFAU1018455', 'ETD': '2023-11-22', 'ETA': '2023-11-25', 'ATD': '2023-11-22', 'ATA': '2023-12-08', 'Container Release': '2023-12-16', 'Container Return': '2023-12-19'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4654708', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'FFAU1078515', 'ETD': '2023-11-22', 'ETA': '2023-11-25', 'ATD': '2023-11-22', 'ATA': '2023-12-08', 'Container Release': '2023-12-16', 'Container Return': '2023-12-19'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4654708', 'From': 'JAKARTA', 'To': 'LAEM CHABANG', 'Container Number': 'SEKU5216423', 'ETD': '2023-11-22', 'ETA': '2023-11-

 25%|██▌       | 2/8 [00:04<00:13,  2.21s/it]

...

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4681560', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'UETU5452263', 'ETD': '2023-12-23', 'ETA': '2023-12-31', 'ATD': '2023-12-23', 'ATA': '2023-12-31'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4699130', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'DFSU7688753', 'ETD': '2024-01-06', 'ETA': '2024-01-11', 'ATD': '2024-01-06'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4699130', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'FFAU4372924', 'ETD': '2024-01-06', 'ETA': '2024-01-11', 'ATD': '2024-01-06'}


 88%|████████▊ | 7/8 [00:15<00:02,  2.50s/it]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4699130', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'TGBU5329165', 'ETD': '2024-01-06', 'ETA': '2024-01-11', 'ATD': '2024-01-06'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4680636', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'Container Number': 'BMOU6243231', 'ETD': '2023-12-31', 'ETA': '2024-01-02', 'ATD': '2023-12-31', 'ATA': '2024-01-06'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4680636', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'Container Number': 'FFAU2589037', 'ETD': '2023-12-31', 'ETA': '2024-01-02', 'ATD': '2023-12-31', 'ATA': '2024-01-06'}


100%|██████████| 8/8 [00:17<00:00,  2.18s/it]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4680636', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'Container Number': 'TXGU5040262', 'ETD': '2023-12-31', 'ETA': '2024-01-02', 'ATD': '2023-12-31', 'ATA': '2024-01-06'}


In [9]:
hasil_akhir

[{'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4653697',
  'From': 'JAKARTA',
  'To': 'LAEM CHABANG',
  'Container Number': 'BMOU6920374',
  'ETD': '2023-11-22',
  'ETA': '2023-11-25',
  'ATD': '2023-11-22',
  'ATA': '2023-12-09',
  'Container Release': '2023-12-19',
  'Container Return': '2023-12-22',
  '_id': ObjectId('659dfe7d6853128966c44393')},
 {'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4653697',
  'From': 'JAKARTA',
  'To': 'LAEM CHABANG',
  'Container Number': 'KMTU9354174',
  'ETD': '2023-11-22',
  'ETA': '2023-11-25',
  'ATD': '2023-11-22',
  'ATA': '2023-12-09',
  'Container Release': '2023-12-19',
  'Container Return': '2023-12-22',
  '_id': ObjectId('659dfe7d6853128966c44394')},
 {'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4653697',
  'From': 'JAKARTA',
  'To': 'LAEM CHABANG',
  'Container Number': 'SEGU4808737',
  'ETD': '2023-11-22',
  'ETA': '2023-11-25',
  'ATD': '2023-11-22',
  'ATA': '2023-12-09',
  'Container Release': '2023-12-19',
  'Container Return': '2023-12-22',
  '_

In [8]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo, update_db

mongo_table_initiation()
insert_many_mongo(hasil_akhir)

Today's Collection Name ===>  all_tracking_Jan-10-2024
Today's Collection Has Been Made
Inserting Many Complete!!


In [9]:
update_db(hasil_akhir)

KeyError: 'Container Return'

In [9]:
gagal

[]

In [7]:
# exporting to excel file
df = pd.DataFrame(hasil_akhir)
excel_file_path = 'export excel/KMTC.xlsx'
df.to_excel(excel_file_path, index=False)